In [ ]:
import pandas as pd
import re
import pickle
import random

In [ ]:
data = pd.read_csv('final_data.csv', index_col=0)
data

In [ ]:
data.loc[0, 'SCOREMARGIN'] = 0
for i in range(1, 124992):
    if pd.isna(data.loc[i, 'SCOREMARGIN']) or data.loc[i, 'SCOREMARGIN'] == 0:
        data.loc[i, 'SCOREMARGIN'] = data.loc[i - 1, 'SCOREMARGIN']
    if data.loc[i, 'PERIOD'] == 1 and data.loc[i-1, 'PERIOD'] == 4:
        data.loc[i, 'SCOREMARGIN'] = 0
data.loc[data['SCOREMARGIN'] == "TIE", 'SCOREMARGIN'] = 0
data

In [ ]:
data.loc[0, 'SCORE'] = "0 - 0"
for i in range(1, 124992):
    if pd.isna(data.loc[i, 'SCORE']) or data.loc[i, 'SCORE'] == "0 - 0":
        data.loc[i, 'SCORE'] = data.loc[i - 1, 'SCORE']
    if data.loc[i, 'PERIOD'] == 1 and data.loc[i-1, 'PERIOD'] == 4:
        data.loc[i, 'SCORE'] = "0 - 0"
        
data

In [ ]:
SCORE_1 = []
SCORE_2 = []
for i in range(124992):
    match = re.match(r"(\d+)\s*-\s*(\d+)", data.loc[i, 'SCORE'])
    if match:
        SCORE_1.append(int(match.group(1)))
        SCORE_2.append(int(match.group(2)))
data['SCORE_1'] = SCORE_1
data['SCORE_2'] = SCORE_2

In [ ]:
data.drop(columns=['SCORE'], inplace=True)

In [ ]:
result = []
for i in range(124991):
    if data.loc[i, 'PERIOD'] == 4 and (data.loc[i+1, 'PERIOD'] == 1):
        if data.loc[i, 'SCORE_1'] > data.loc[i, 'SCORE_2']:
            result.append(data.loc[i, 'TEAM_1'])
        else:
            result.append(data.loc[i, 'TEAM_2'])
if data.loc[124991, 'SCORE_1'] > data.loc[124991, 'SCORE_2']:
    result.append(data.loc[124991, 'TEAM_1'])
else:
    result.append(data.loc[124991, 'TEAM_2'])
result

In [ ]:
j = 0
data['WINNER'] = pd.Series()
for i in range(124991):
    data.loc[i, 'WINNER'] = result[j]
    if data.loc[i, 'PERIOD'] == 4 and (data.loc[i+1, 'PERIOD'] == 1):
        j+=1
data.loc[124991, "WINNER"] = result[j]
data

In [ ]:
data.loc[data['PERIOD'] > 1,'TIME_REMAINING'] += 720*(data['PERIOD']-1)
data

In [ ]:
# data.to_csv('final_data.csv')


In [ ]:
data = pd.read_csv('final_data.csv', index_col=0)
data

In [ ]:
data.SCOREMARGIN = -1*data.SCOREMARGIN

In [ ]:
grouped = data.groupby(['GAME_ID'])
grouped_dataframes = [group for _, group in grouped]
random.shuffle(grouped_dataframes)
grouped_dataframes[263]

In [ ]:
with open('train_set.pkl', 'wb') as file:
    pickle.dump(grouped_dataframes[:184], file)
with open('test_set.pkl', 'wb') as file:
    pickle.dump(grouped_dataframes[184:], file)

In [ ]:
with open('test_set.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

# Access the DataFrames
loaded_list[0]